In [ ]:
%load_ext autoreload
%autoreload 2

%connect_info

In [ ]:
import django

In [ ]:
django.setup()

In [ ]:
"""
Project Configuration Sheets
"""

FIN_MAN_PROJECT_SHEET_NAME = "Finman Project Configuration"
FIN_MAN_CONFIGURATION_SHEET_NAME="FinMan Configuration_v4 Dev Loading"
VENDOR_CONFIGURATION_SHEET_NAME = "Vendor Configuration_v1 - Test"

In [ ]:
"""
Populate project and project roles
"""

from django.db import transaction

@transaction.atomic()
def populate_project_and_roles():
    from ib_iam.populate.populate_utils import populate_projects
    populate_projects(spread_sheet_name=FIN_MAN_PROJECT_SHEET_NAME)

    from ib_tasks.populate.population_utils import populate_project_roles
    populate_project_roles(FIN_MAN_CONFIGURATION_SHEET_NAME)

In [ ]:
"""
Populate tasks related data
"""

from django.db import transaction

@transaction.atomic()
def populate_task_and_boards_related_data():
    
    configuration_sheets = [
        FIN_MAN_CONFIGURATION_SHEET_NAME
    ]

    for configuration_sheet in configuration_sheets:

        from ib_tasks.populate.population_utils import populate_data
        populate_data(configuration_sheet)
        
        from ib_boards.populate.get_data_from_sheet_for_boards_and_columns import GetBoardsAndColumnsDataFromSheet
        loader = GetBoardsAndColumnsDataFromSheet()
        loader.get_data_from_sheet(configuration_sheet)
        loader.create_boards_and_columns(configuration_sheet)

In [ ]:
"""
Populate project related data
"""
from django.db import transaction

@transaction.atomic()
def populate_project_related_data():

    from ib_tasks.populate.population_utils import populate_projects_for_task_templates
    populate_projects_for_task_templates(FIN_MAN_PROJECT_SHEET_NAME)
    
    from ib_boards.populate.get_data_from_sheet_for_project_boards import GetSheetDataForProjectBoards
    loader = GetSheetDataForProjectBoards()
    loader.get_data_from_project_boards_sub_sheet(FIN_MAN_PROJECT_SHEET_NAME)

In [ ]:
populate_project_and_roles()
populate_task_and_boards_related_data()
populate_project_related_data()

In [ ]:
from ib_tasks.populate.write_stage_actions import writing_data_to_stage_actions_logic
writing_data_to_stage_actions_logic()

In [ ]:
"""
Populate Ib IAM Related Data For FinMan Project Configuration

Note:- While Running the this below script need to Run the ELASTIC SEARCH SERVER
"""
from ib_iam.populate.populate_utils import (
    INITIAL_USERS, COMPANIES, TEAMS, TEAM_USERS_LEVELS, LEVELS_WISE_USERS, SUPERIOR_WITH_SUBORDINATE_USERS
)

from ib_iam.populate.populate_utils import (
    populate_companies_and_teams, populate_initial_users, populate_levels
)
populate_companies_and_teams(companies=COMPANIES, teams=TEAMS)
populate_initial_users(users=INITIAL_USERS)
team_name = "Tech Team"
populate_levels(
    team_name=team_name, team_user_levels=TEAM_USERS_LEVELS,levels_wise_users=LEVELS_WISE_USERS,
    superior_with_subordinate_users=SUPERIOR_WITH_SUBORDINATE_USERS
)

from ib_users.models import *
UserAccount.objects.create_superuser(username='admin', password='admin@123', email='admin@example.com')
UserAccount.objects.create_superuser(username='fduser', password='fduser@123', email='fduser@example.com')

In [ ]:
"""
Populate Elastic Search Data For Cities

"""
from ib_iam.populate.populate_elastic_search_data import populate_data
populate_data()

In [ ]:
from elasticsearch_dsl import connections
from django.conf import settings
es = connections.create_connection(
hosts=[settings.ELASTICSEARCH_ENDPOINT], timeout=20)

In [ ]:
from ib_tasks.documents.elastic_task import TASK_INDEX_NAME
es.indices.create(index = TASK_INDEX_NAME)

In [ ]:
"""
Populate Users with company and roles and teams

"""


from ib_iam.populate.populate_complete_user_details_bulk import populate_complete_user_details_bulk, USERS

populate_complete_user_details_bulk(users=USERS)